In [ ]:
import time
import cudaq
import numpy as np

# 2. 큐비트 수 및 파라미터 설정
qubit_count = 30
layer_count = 2

total_params = layer_count * qubit_count
initial_params = np.random.uniform(-np.pi, np.pi, total_params).tolist()

@cudaq.kernel
def mps_circuit(params: list[float]):
    qubits = cudaq.qvector(qubit_count)
    param_idx = 0

    for l in range(layer_count):
        for i in range(qubit_count):
            ry(params[param_idx], qubits[i])
            param_idx += 1
        
        for i in range(qubit_count - 1):
            cx(qubits[i], qubits[i+1])

print("1. NVIDIA GPU")
cudaq.set_target("nvidia")
s = time.time()
for _ in range(30):
    r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
e = time.time()
print(f"   Elapsed time: {e - s:.4f}s")
print(f"   Result: {r}")

# TensorNet Full Contraction 실행
print("2. TensorNet (Full Contraction)")
cudaq.set_target("tensornet")
s = time.time()
for _ in range(30):
    r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
e = time.time()
print(f"   Elapsed time: {e - s:.4f}s")
print(f"   Result: {r}\n")

# TensorNet MPS 실행 (더 메모리 효율적)
print("3. TensorNet MPS (Matrix Product State)")
cudaq.set_target("tensornet-mps")
s = time.time()
for _ in range(30):
    r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
e = time.time()
print(f"   Elapsed time: {e - s:.4f}s")
print(f"   Result: {r}\n")

1. NVIDIA GPU (cuStateVec FP64)
   Elapsed time: 22.6124s
   Result: -0.4658743243202536
2. TensorNet (Full Contraction)
   Elapsed time: 0.6061s
   Result: -0.46587421381013505

3. TensorNet MPS (Matrix Product State)
   Elapsed time: 6.3675s
   Result: -0.4658742138101219



In [ ]:
import time
import cudaq
import numpy as np

# 절대 한계 찾기
for qubit_count in [60, 70, 80, 90]:
    print(f"\n{'='*70}")
    print(f"Qubit: {qubit_count} | 2^{qubit_count} = {2**qubit_count / (1024**6):.1e} EB")
    print(f"{'='*70}")
    
    layer_count = 2
    total_params = layer_count * qubit_count
    initial_params = np.random.uniform(-np.pi, np.pi, total_params).tolist()

    @cudaq.kernel
    def mps_circuit(params: list[float]):
        qubits = cudaq.qvector(qubit_count)
        param_idx = 0

        for l in range(layer_count):
            for i in range(qubit_count):
                ry(params[param_idx], qubits[i])
                param_idx += 1
            
            for i in range(qubit_count - 1):
                cx(qubits[i], qubits[i+1])

    # TensorNet Full Contraction
    print("FC: ", end="", flush=True)
    try:
        cudaq.set_target("tensornet")
        s = time.time()
        r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
        e = time.time()
        print(f"✓ {e-s:.4f}s | Result: {r:.6f}")
    except Exception as ex:
        print(f"✗ {type(ex).__name__}: OOM?")

    # TensorNet MPS
    print("MPS: ", end="", flush=True)
    try:
        cudaq.set_target("tensornet-mps")
        s = time.time()
        r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
        e = time.time()
        print(f"✓ {e-s:.4f}s | Result: {r:.6f}")
    except Exception as ex:
        print(f"✗ {type(ex).__name__}")


Qubit: 60 | 2^60 = 1.0e+00 EB
FC: ✓ 0.0800s | Result: 0.528468
MPS: ✓ 0.5157s | Result: 0.528468

Qubit: 70 | 2^70 = 1.0e+03 EB
FC: ✓ 0.0586s | Result: 0.746481
MPS: ✓ 0.4618s | Result: 0.746481

Qubit: 80 | 2^80 = 1.0e+06 EB
FC: ✓ 0.0578s | Result: 0.405645
MPS: ✓ 0.5138s | Result: 0.405645

Qubit: 90 | 2^90 = 1.1e+09 EB
FC: ✓ 0.0603s | Result: -0.393240
MPS: ✓ 0.5759s | Result: -0.393240


# cuPauliProp 사용 방법

**cuPauliProp (cuQuantum의 일부):**
- Pauli operator propagation에 최적화
- Clifford 회로 시뮬레이션에 매우 효율적
- **C++ 직접 사용 필요**

**CUDA-Q와의 차이:**
- CUDA-Q: High-level Python API, 범용 시뮬레이션
- cuPauliProp: Low-level C++ API, Pauli 특화

**생성된 파일:**
1. `pauli_prop_example.cpp` - cuPauliProp 사용 예제
2. `compile_pauli_prop.sh` - 컴파일 스크립트

**컴파일 및 실행:**
```bash
chmod +x compile_pauli_prop.sh
./compile_pauli_prop.sh
./pauli_prop_example
```

**주요 기능:**
- Pauli 문자열 표현 및 전파
- Clifford gate (H, S, CNOT 등) 적용
- Pauli rotation gate
- 노이즈 채널 시뮬레이션

In [11]:
# CUDA-Q 버전 확인
print("CUDA-Q version:", cudaq.__version__)

# 사용 가능한 백엔드 확인
print("\nAvailable targets:")
targets = cudaq.get_targets()
for target in targets:
    print(f"  - {target}")

CUDA-Q version: CUDA-Q Version 0.12.0 (https://github.com/NVIDIA/cuda-quantum 6adf92bcda4df7465e4fe82f1c8f782ae69d8bd2)

Available targets:
  - Target tensornet-mps
	simulator=tensornet_mps
	platform=default
	description=cutensornet simulator backend target based on matrix product state representation
	precision=fp64
Supported Arguments:
  - option (Specify the target options as a comma-separated list.
Supported options are 'fp32', 'fp64')

  - Target stim
	simulator=stim
	platform=default
	description=Stim-based CPU-only backend target
	precision=fp64

  - Target quera
	simulator=qpp
	platform=default
	description=CUDA-Q target for QuEra.
	precision=fp32
Supported Arguments:
  - machine (Specify the QuEra QPU.)
  - default_bucket (Specify a default S3 bucket for QuEra results.)

  - Target quantum_machines
	simulator=qpp
	platform=default
	description=CUDA-Q target for Quantum Machines.
	precision=fp32
Supported Arguments:
  - url (Specify Quantum Machine base server url.)
  - executo

In [12]:

# tensornet 백엔드의 설정 확인
cudaq.set_target("tensornet")
target_info = cudaq.get_target()
print(f"\nCurrent target: {target_info}")
print(f"Target description: {target_info.description if hasattr(target_info, 'description') else 'N/A'}")

# tensornet 관련 환경변수 확인
import os
print("\nEnvironment variables related to tensornet/cuQuantum:")
for key, value in os.environ.items():
    if 'TENSOR' in key.upper() or 'CUDA' in key.upper() or 'CUTENSOR' in key.upper():
        print(f"  {key}={value}")


Current target: Target tensornet
	simulator=tensornet
	platform=default
	description=cutensornet simulator backend target based on full tensor network contraction
	precision=fp64
Supported Arguments:
  - option (Specify the target options as a comma-separated list.
Supported options are 'fp32', 'fp64')

Target description: cutensornet simulator backend target based on full tensor network contraction

Environment variables related to tensornet/cuQuantum:
  CUDAQ_DYNLIBS=/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/cuquantum/lib/libcustatevec.so.1:/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/cuquantum/lib/libcutensornet.so.2:/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/cuquantum/lib/libcudensitymat.so.0:/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/cutensor/lib/libcutensor.so.2:/home/ubuntu/miniconda3/envs/lqgan/lib/python3.11/site-packages/nvidia/cuda_runtime/lib/libcudart.so.12:/home/ubuntu/miniconda3/envs/lqga

In [22]:
import time
import cudaq
import numpy as np

# 1. 텐서 네트워크 백엔드 설정 (MPS 시뮬레이션 최적화)
cudaq.set_target("nvidia")


@cudaq.kernel
def mps_circuit(params: list[float]):
    qubits = cudaq.qvector(qubit_count)
    param_idx = 0

    # 레이어 반복
    for l in range(layer_count):
        # (1) Single-qubit Rotation Layer
        for i in range(qubit_count):
            ry(params[param_idx], qubits[i])
            param_idx += 1
        
        # (2) Entanglement Layer (Nearest-Neighbor / MPS Style)
        for i in range(qubit_count - 1):
            cx(qubits[i], qubits[i+1])

# 4. 임의의 파라미터 생성 및 실행
# 필요한 파라미터 수: 레이어 수 * 큐비트 수


# 회로 실행 및 결과 확인
s = time.time()
for _ in range(30):
    r = cudaq.observe(mps_circuit, cudaq.spin.z(0), initial_params).expectation()
e = time.time()
print("Elapsed time for 30 executions:", e - s)
print(r)

RuntimeError: requested size is too big